In [5]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import numpy as np


In [8]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Flatten images to apply PCA (original size: 32x32x3)
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

# Standardize the data to zero mean and unit variance
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train_flat)
x_test_scaled = scaler.transform(x_test_flat)


In [3]:
# Flatten images to apply PCA (original size: 32x32x3)
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

In [6]:
# Standardize the data to zero mean and unit variance
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train_flat)

In [9]:
# Apply PCA with 150 components
pca = PCA(n_components=150)
x_train_pca = pca.fit_transform(x_train_scaled)
x_test_pca = pca.transform(x_test_scaled)

# Check variance explained by each component
explained_variance_ratio = np.sum(pca.explained_variance_ratio_)
print(f"Total Explained Variance with 150 components: {explained_variance_ratio:.4f}")


Total Explained Variance with 150 components: 0.9271


In [10]:
def build_model(input_dim):
    model = Sequential([
        Dense(512, activation='relu', input_shape=(input_dim,)),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')  # 10 classes for CIFAR-10
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [11]:
# Build and train the model on original dataset
model_original = build_model(x_train_flat.shape[1])
history_original = model_original.fit(x_train_flat, y_train, epochs=10, batch_size=64, validation_split=0.2)

# Evaluate on test data
test_loss_orig, test_accuracy_orig = model_original.evaluate(x_test_flat, y_test)
print(f"Original Data Test Accuracy: {test_accuracy_orig:.4f}")


Epoch 1/10
625/625 [==============================] - 25s 37ms/step - loss: 25.4386 - accuracy: 0.2121 - val_loss: 3.0805 - val_accuracy: 0.2449
Epoch 2/10
625/625 [==============================] - 23s 37ms/step - loss: 2.4441 - accuracy: 0.2858 - val_loss: 2.3814 - val_accuracy: 0.2864
Epoch 3/10
625/625 [==============================] - 23s 37ms/step - loss: 2.0618 - accuracy: 0.3147 - val_loss: 1.9876 - val_accuracy: 0.3235
Epoch 4/10
625/625 [==============================] - 23s 37ms/step - loss: 1.9173 - accuracy: 0.3418 - val_loss: 1.8720 - val_accuracy: 0.3545
Epoch 5/10
625/625 [==============================] - 23s 37ms/step - loss: 1.8502 - accuracy: 0.3509 - val_loss: 1.9683 - val_accuracy: 0.3098
Epoch 6/10
625/625 [==============================] - 23s 37ms/step - loss: 1.8243 - accuracy: 0.3569 - val_loss: 1.8860 - val_accuracy: 0.3310
Epoch 7/10
625/625 [==============================] - 24s 38ms/step - loss: 1.8067 - accuracy: 0.3615 - val_loss: 1.8392 - val_accuracy

In [12]:
# Build and train the model on PCA-reduced dataset
model_pca = build_model(x_train_pca.shape[1])
history_pca = model_pca.fit(x_train_pca, y_train, epochs=10, batch_size=64, validation_split=0.2)

# Evaluate on test data
test_loss_pca, test_accuracy_pca = model_pca.evaluate(x_test_pca, y_test)
print(f"PCA Reduced Data Test Accuracy: {test_accuracy_pca:.4f}")


Epoch 1/10
625/625 [==============================] - 9s 10ms/step - loss: 1.6672 - accuracy: 0.4117 - val_loss: 1.5289 - val_accuracy: 0.4640
Epoch 2/10
625/625 [==============================] - 6s 10ms/step - loss: 1.3667 - accuracy: 0.5171 - val_loss: 1.4317 - val_accuracy: 0.4973
Epoch 3/10
625/625 [==============================] - 6s 10ms/step - loss: 1.2051 - accuracy: 0.5740 - val_loss: 1.4082 - val_accuracy: 0.5179
Epoch 4/10
625/625 [==============================] - 6s 10ms/step - loss: 1.0710 - accuracy: 0.6219 - val_loss: 1.4094 - val_accuracy: 0.5205
Epoch 5/10
625/625 [==============================] - 6s 10ms/step - loss: 0.9410 - accuracy: 0.6683 - val_loss: 1.4578 - val_accuracy: 0.5298
Epoch 6/10
625/625 [==============================] - 6s 10ms/step - loss: 0.8234 - accuracy: 0.7072 - val_loss: 1.5006 - val_accuracy: 0.5232
Epoch 7/10
625/625 [==============================] - 6s 10ms/step - loss: 0.7096 - accuracy: 0.7491 - val_loss: 1.6034 - val_accuracy: 0.5285

In [13]:
# Calculate and display trade-off results
accuracy_difference = (test_accuracy_orig - test_accuracy_pca) * 100
print("\n--- Dimensionality Reduction Analysis ---")
print(f"Original Dataset Dimensionality: {x_train_flat.shape[1]}")
print(f"Reduced Dataset Dimensionality: {x_train_pca.shape[1]}")
print(f"Accuracy with Original Dataset: {test_accuracy_orig:.4f}")
print(f"Accuracy with PCA Reduced Dataset: {test_accuracy_pca:.4f}")
print(f"Accuracy Difference: {accuracy_difference:.2f}%")



--- Dimensionality Reduction Analysis ---
Original Dataset Dimensionality: 3072
Reduced Dataset Dimensionality: 150
Accuracy with Original Dataset: 0.3315
Accuracy with PCA Reduced Dataset: 0.5110
Accuracy Difference: -17.95%
